In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
!pip install transformers
!pip install datasets
!pip install wandb
!pip install -U datasets
!pip install objaverse
!pip install diffusers
!pip install trimesh
!pip install jaxtyping
!pip install pytorch-lightning
!pip install ijson
!pip install triton==3.2.0
!pip install wandb
!pip install decord
!pip uninstall -y pillow
!pip install pillow==9.5.0 --no-cache-dir
!pip install yt-dlp opencv-python accelerate
!pip install "numpy<2.0"

In [ ]:
!pip install torch-cluster -f https://data.pyg.org/whl/torch-$(python -c "import torch; print(torch.__version__)").html
!apt-get update && apt-get install -y libaio-dev

In [ ]:
import sys
!rm -rf AI-Game-Engine
!git clone https://github.com/Sakib323/AI-Game-Engine.git
sys.path.append('/kaggle/working/AI-Game-Engine')
from mmfreelm.models.hgrn_bit.video_gen import VideoDiT_models

In [ ]:
import os
import ast
import torch
import cv2
import numpy as np
from datasets import load_dataset
from yt_dlp import YoutubeDL
from transformers import AutoTokenizer
from diffusers import AutoencoderKLTemporalDecoder
from torchvision import transforms

# --- Configuration ---
DATASET_NAME = "multimodalart/panda-70m"
SPLIT = "test"
TOKENIZER_NAME = "Sakib323/MMfreeLM-370M"
OUTPUT_DIR = "./processed_data"
VAE_MODEL = "stabilityai/stable-video-diffusion-img2vid-xt"
MAX_FRAMES = 16 
HEIGHT = 576
WIDTH = 1024

# Create output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)

# --- 1. Load Models ---
print("Loading Models...")
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_NAME)
tokenizer.pad_token = tokenizer.eos_token

# Load VAE
vae = AutoencoderKLTemporalDecoder.from_pretrained(
    VAE_MODEL, 
    subfolder="vae", 
    torch_dtype=torch.float16 if device == "cuda" else torch.float32
).to(device)
vae.eval()

# --- 2. Helper Functions ---

def parse_timestamp(ts_str):
    parts = ts_str.split(':')
    seconds = 0.0
    for part in parts:
        seconds = seconds * 60 + float(part)
    return seconds

def preprocess_video(video_path, max_frames=16, height=576, width=1024):
    """Reads video, ensures frame count, and returns (T, 3, H, W)."""
    cap = cv2.VideoCapture(video_path)
    frames = []
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(frame)
        # Stop early if we have enough frames to save processing time
        if len(frames) >= max_frames:
            break
            
    cap.release()
    
    if len(frames) < 1:
        return None
    
    video_np = np.array(frames) 
    video_tensor = torch.tensor(video_np).permute(0, 3, 1, 2)
    
    transform = transforms.Compose([
        transforms.Resize((height, width)),
        transforms.Lambda(lambda x: x / 127.5 - 1.0)
    ])
    
    processed_frames = torch.stack([transform(f) for f in video_tensor])
    
    return processed_frames

def download_clip(url, start_time, end_time, output_filename):
    ydl_opts = {
        'format': 'bestvideo[height<=576][ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best',
        'outtmpl': output_filename,
        'quiet': True,
        'no_warnings': True,
        # Fix for the YouTube extractor warnings
        'extractor_args': {'youtube': {'player_client': ['default', 'ios']}},
        'external_downloader': 'ffmpeg',
        'external_downloader_args': {
            'ffmpeg_i': ['-ss', str(start_time), '-to', str(end_time)]
        }
    }
    
    try:
        with YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
        return True
    except Exception as e:
        # print(f"Download error: {e}")
        return False

# --- 3. Main Processing Loop ---

print(f"Streaming dataset {DATASET_NAME}...")
dataset = load_dataset(DATASET_NAME, split=SPLIT, streaming=True)

for i, row in enumerate(dataset):
    try:
        video_id = row['videoID']
        url = row['url']
        
        timestamps = ast.literal_eval(row['timestamp'])
        captions = ast.literal_eval(row['caption'])
        
        for idx, (time_range, caption) in enumerate(zip(timestamps, captions)):
            start_str, end_str = time_range
            start_sec = parse_timestamp(start_str)
            end_sec = parse_timestamp(end_str)
            
            clip_name = f"{video_id}_{idx}"
            video_filename = f"{clip_name}.mp4"
            save_path = os.path.join(OUTPUT_DIR, f"{clip_name}.pt")
            
            if os.path.exists(save_path):
                continue

            success = download_clip(url, start_sec, end_sec, video_filename)
            
            if success and os.path.exists(video_filename):
                try:
                    # B. Process Video
                    # Returns shape (T, 3, H, W)
                    pixel_values = preprocess_video(video_filename, max_frames=MAX_FRAMES, height=HEIGHT, width=WIDTH)
                    
                    if pixel_values is not None:
                        # Move to GPU
                        pixel_values = pixel_values.to(device, dtype=torch.float16 if device=="cuda" else torch.float32)
                        
                        with torch.no_grad():
                            latents = vae.encode(pixel_values).latent_dist.mode()
                        text_tokens = tokenizer(
                            caption, 
                            padding="max_length", 
                            truncation=True, 
                            max_length=128, 
                            return_tensors="pt"
                        )
                        
                        # D. Save
                        data_point = {
                            "video_latents": latents.cpu(), # (T, 4, h, w)
                            "input_ids": text_tokens["input_ids"].cpu(),
                            "attention_mask": text_tokens["attention_mask"].cpu(),
                            "caption": caption
                        }
                        torch.save(data_point, save_path)
                        print(f"Processed: {clip_name} | Frames: {pixel_values.shape[0]}")

                except Exception as e:
                    print(f"Error processing {clip_name}: {e}")
                
                finally:
                    if os.path.exists(video_filename):
                        os.remove(video_filename)
            
    except Exception as e:
        print(f"Skipping row {i}: {e}")
        continue

2025-11-22 13:58:57.454739: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763819937.476966    1143 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763819937.483683    1143 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

Processed: FaJAVdXHxkA_2 | Frames: 16
Processed: FiLn6AhMqS0_0 | Frames: 16                   
Processed: FiLn6AhMqS0_1 | Frames: 16                   
Processed: FiLn6AhMqS0_2 | Frames: 16                    
Processed: Fmf9jNwPkCQ_0 | Frames: 16                  
Processed: Fmf9jNwPkCQ_1 | Frames: 16                  
Processed: Fmf9jNwPkCQ_2 | Frames: 16                  
Processed: Fn3lCshxrjQ_0 | Frames: 16                   
Processed: Fn3lCshxrjQ_1 | Frames: 16                   
Processed: Fn3lCshxrjQ_2 | Frames: 16                   
Processed: Fnw678SNsyI_0 | Frames: 16                 
Processed: Fnw678SNsyI_1 | Frames: 16                 
Processed: Fnw678SNsyI_2 | Frames: 16                 
Processed: Fsxln-bIF6Q_0 | Frames: 16                   
Processed: Fsxln-bIF6Q_1 | Frames: 16                 
Processed: Fsxln-bIF6Q_2 | Frames: 16                   
Processed: FubmRYr-9nQ_0 | Frames: 16                   
Processed: FubmRYr-9nQ_1 | Frames: 16                   
Pro

[av1 @ 0x53fb2dc0] Your platform doesn't suppport hardware accelerated AV1 decoding.
[av1 @ 0x53fb2dc0] Failed to get pixel format.
[av1 @ 0x53fb2dc0] Missing Sequence Header.
[av1 @ 0x53fb2dc0] Missing Sequence Header.
[av1 @ 0x53fb2dc0] Missing Sequence Header.
[av1 @ 0x53fb2dc0] Missing Sequence Header.
[av1 @ 0x53fb2dc0] Missing Sequence Header.
[av1 @ 0x53fb2dc0] Missing Sequence Header.
[av1 @ 0x53fb2dc0] Missing Sequence Header.
[av1 @ 0x53fb2dc0] Missing Sequence Header.
[av1 @ 0x53fb2dc0] Missing Sequence Header.
[av1 @ 0x53fb2dc0] Missing Sequence Header.
[av1 @ 0x53fb2dc0] Missing Sequence Header.
[av1 @ 0x53fb2dc0] Missing Sequence Header.
[av1 @ 0x53fb2dc0] Missing Sequence Header.
[av1 @ 0x53fb2dc0] Missing Sequence Header.
[av1 @ 0x53fb2dc0] Missing Sequence Header.
[av1 @ 0x53fb2dc0] Missing Sequence Header.
[av1 @ 0x53fb2dc0] Missing Sequence Header.
[av1 @ 0x53fb2dc0] Missing Sequence Header.
[av1 @ 0x53fb2dc0] Missing Sequence Header.
[av1 @ 0x53fb2dc0] Missing Seque

[av1 @ 0x53fc4bc0] Your platform doesn't suppport hardware accelerated AV1 decoding.
[av1 @ 0x53fc4bc0] Failed to get pixel format.
[av1 @ 0x53fc4bc0] Missing Sequence Header.
[av1 @ 0x53fc4bc0] Missing Sequence Header.
[av1 @ 0x53fc4bc0] Missing Sequence Header.
[av1 @ 0x53fc4bc0] Missing Sequence Header.
[av1 @ 0x53fc4bc0] Missing Sequence Header.
[av1 @ 0x53fc4bc0] Missing Sequence Header.
[av1 @ 0x53fc4bc0] Missing Sequence Header.
[av1 @ 0x53fc4bc0] Missing Sequence Header.
[av1 @ 0x53fc4bc0] Missing Sequence Header.
[av1 @ 0x53fc4bc0] Missing Sequence Header.
[av1 @ 0x53fc4bc0] Missing Sequence Header.
[av1 @ 0x53fc4bc0] Missing Sequence Header.
[av1 @ 0x53fc4bc0] Missing Sequence Header.
[av1 @ 0x53fc4bc0] Missing Sequence Header.
[av1 @ 0x53fc4bc0] Missing Sequence Header.
[av1 @ 0x53fc4bc0] Missing Sequence Header.
[av1 @ 0x53fc4bc0] Missing Sequence Header.
[av1 @ 0x53fc4bc0] Missing Sequence Header.
[av1 @ 0x53fc4bc0] Missing Sequence Header.
[av1 @ 0x53fc4bc0] Missing Seque

[av1 @ 0x56f06380] Your platform doesn't suppport hardware accelerated AV1 decoding.
[av1 @ 0x56f06380] Failed to get pixel format.
[av1 @ 0x56f06380] Missing Sequence Header.
[av1 @ 0x56f06380] Missing Sequence Header.
[av1 @ 0x56f06380] Missing Sequence Header.
[av1 @ 0x56f06380] Missing Sequence Header.
[av1 @ 0x56f06380] Missing Sequence Header.
[av1 @ 0x56f06380] Missing Sequence Header.
[av1 @ 0x56f06380] Missing Sequence Header.
[av1 @ 0x56f06380] Missing Sequence Header.
[av1 @ 0x56f06380] Missing Sequence Header.
[av1 @ 0x56f06380] Missing Sequence Header.
[av1 @ 0x56f06380] Missing Sequence Header.
[av1 @ 0x56f06380] Missing Sequence Header.
[av1 @ 0x56f06380] Missing Sequence Header.
[av1 @ 0x56f06380] Missing Sequence Header.
[av1 @ 0x56f06380] Missing Sequence Header.
[av1 @ 0x56f06380] Missing Sequence Header.
[av1 @ 0x56f06380] Missing Sequence Header.
[av1 @ 0x56f06380] Missing Sequence Header.
[av1 @ 0x56f06380] Missing Sequence Header.
[av1 @ 0x56f06380] Missing Seque

Processed: FxnrYU1sxXw_0 | Frames: 16                   
Processed: FxnrYU1sxXw_1 | Frames: 16                   
Processed: FxnrYU1sxXw_2 | Frames: 16                   
Processed: G1NxZ7XO5Ko_0 | Frames: 16                  
Processed: G1NxZ7XO5Ko_1 | Frames: 16                   
Processed: G1NxZ7XO5Ko_2 | Frames: 16                   
Processed: G5J6BfFMZPM_0 | Frames: 16                  
Processed: G5J6BfFMZPM_1 | Frames: 16                   


In [ ]:
import os
import torch
from tqdm import tqdm
import math

def merge_pt_files(source_dir, output_dir, chunk_size=500):
    if not os.path.exists(source_dir):
        print(f"Error: Source directory '{source_dir}' not found.")
        return

    os.makedirs(output_dir, exist_ok=True)
    all_files = [f for f in os.listdir(source_dir) if f.endswith('.pt')]
    all_files.sort()
    
    total_files = len(all_files)
    print(f"Found {total_files} .pt files in {source_dir}")
    
    if total_files == 0:
        print("No files to merge.")
        return
    num_chunks = math.ceil(total_files / chunk_size)
    print(f"Merging into {num_chunks} files (Limit: {chunk_size} files per chunk)...")

    for i in range(num_chunks):
        start_idx = i * chunk_size
        end_idx = min((i + 1) * chunk_size, total_files)
        
        batch_files = all_files[start_idx:end_idx]
        merged_data = []
        
        print(f"Processing chunk {i+1}/{num_chunks} (Files {start_idx} to {end_idx-1})...")
        
        for filename in tqdm(batch_files, desc=f"Loading Chunk {i+1}"):
            file_path = os.path.join(source_dir, filename)
            try:
                data = torch.load(file_path)
                if isinstance(data, list):
                    merged_data.extend(data)
                else:
                    merged_data.append(data)
                    
            except Exception as e:
                print(f"Failed to load {filename}: {e}")

        output_filename = f"merged_chunk_{i}.pt"
        output_path = os.path.join(output_dir, output_filename)
        
        print(f"Saving {len(merged_data)} items to {output_path}...")
        torch.save(merged_data, output_path)
        
        print(f"Deleting processed source files...")
        for filename in batch_files:
            try:
                os.remove(os.path.join(source_dir, filename))
            except OSError as e:
                print(f"Error deleting {filename}: {e}")
        
        del merged_data
        
    print("\nMerge complete! Files saved to:", output_dir)

if __name__ == "__main__":
    SOURCE_DIR = "processed_data" 
    OUTPUT_DIR = "/kaggle/working/merged_output"
    CHUNK_SIZE = 500
    merge_pt_files(SOURCE_DIR, OUTPUT_DIR, CHUNK_SIZE)

In [ ]:
import os
import json
from kaggle.api.kaggle_api_extended import KaggleApi

KAGGLE_USERNAME = "sakibahmed2022"
KAGGLE_KEY = "d0ec7fe8091a3906c6995568c11e9e58" 

os.environ['KAGGLE_USERNAME'] = KAGGLE_USERNAME
os.environ['KAGGLE_KEY'] = KAGGLE_KEY

api = KaggleApi()
api.authenticate()

SOURCE_DIR = "/kaggle/working/merged_output"
DATASET_SLUG = f"{KAGGLE_USERNAME}/video-gen-training-data"
DATASET_TITLE = "Video Gen Training Data"

print(f"✅ Kaggle API AUTHENTICATED!")
print(f"👤 Username: {KAGGLE_USERNAME}")
print(f"📁 Target Dataset: {DATASET_SLUG}")

def create_metadata(source_dir, title, slug):
    """Generates the required dataset-metadata.json file"""
    metadata = {
        "title": title,
        "id": slug,
        "licenses": [{"name": "CC0-1.0"}]
    }
    meta_path = os.path.join(source_dir, "dataset-metadata.json")
    with open(meta_path, 'w') as f:
        json.dump(metadata, f, indent=4)
    print(f"📝 Metadata file created at: {meta_path}")

def robust_upload():

    create_metadata(SOURCE_DIR, DATASET_TITLE, DATASET_SLUG)
    
    print(f"🚀 Starting Upload for directory: {SOURCE_DIR}")
    
    try:
        print("   Attempting to create NEW dataset...")
        api.dataset_create_new(
            folder=SOURCE_DIR,
            public=False, # Private by default
            quiet=False,
            dir_mode='zip' # Zips files for faster upload/less timeout risk
        )
        print(f"🎉 SUCCESS: New dataset created!")
        
    except Exception as e:
        error_str = str(e)
        if "already exists" in error_str or "409" in error_str:
            print(f"⚠️ Dataset already exists. Switching to UPDATE mode...")
            try:
                api.dataset_create_version(
                    folder=SOURCE_DIR,
                    version_notes="Updated merged .pt chunks",
                    quiet=False,
                    dir_mode='zip',
                    delete_old_versions=True
                )
                print(f"🎉 SUCCESS: Dataset version updated!")
            except Exception as ver_e:
                print(f"❌ FAILED to update version: {ver_e}")
        else:
            print(f"❌ FAILED to create dataset: {e}")

    print(f"🔗 Link: https://www.kaggle.com/datasets/{DATASET_SLUG}")

if __name__ == "__main__":
    if os.path.exists(SOURCE_DIR) and len(os.listdir(SOURCE_DIR)) > 0:
        robust_upload()
    else:
        print(f"❌ Error: Source directory '{SOURCE_DIR}' does not exist or is empty.")

In [ ]:
import os
import gc
import glob
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.cuda.amp import GradScaler, autocast
from transformers import AutoTokenizer
from tqdm import tqdm

# Import the loss from your NEW video_gen file
from video_gen import VideoDiT_models, flow_matching_loss

# --- Memory Optimization Config ---
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
torch.cuda.empty_cache()
gc.collect()

# --- Configuration ---
DATA_DIR = "/kaggle/working/merged_output"
MODEL_SAVE_DIR = "/kaggle/working/checkpoints"
TOKENIZER_NAME = "Sakib323/MMfreeLM-370M"

BATCH_SIZE = 1 
GRADIENT_ACCUMULATION_STEPS = 4
LEARNING_RATE = 1e-4
NUM_EPOCHS = 5
NUM_WORKERS = 2

# Dimensions
INPUT_SIZE = (16, 72, 128) 
PATCH_SIZE = (1, 2, 2)

device = "cuda" if torch.cuda.is_available() else "cpu"
os.makedirs(MODEL_SAVE_DIR, exist_ok=True)

# --- 1. Dataset Class (UPDATED) ---
class VideoLatentDataset(Dataset):
    def __init__(self, data_dir):
        self.data = []
        # Scaling factor for StabilityAI VAEs
        self.scale_factor = 0.18215 
        print(f"Loading merged datasets from {data_dir}...")
        files = sorted(glob.glob(os.path.join(data_dir, "*.pt")))
        if not files:
            raise ValueError(f"No .pt files found in {data_dir}")

        for f_path in tqdm(files, desc="Loading chunks"):
            try:
                chunk_data = torch.load(f_path, map_location="cpu")
                if isinstance(chunk_data, list):
                    self.data.extend(chunk_data)
                else:
                    self.data.append(chunk_data)
            except Exception as e:
                print(f"Error loading {f_path}: {e}")
        
        print(f"Total samples loaded: {len(self.data)}")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        latents = item["video_latents"].float()
        
        # 1. Permute: (T, C, H, W) -> (C, T, H, W)
        latents = latents.permute(1, 0, 2, 3) 
        
        # 2. SCALE THE LATENTS (Crucial Change)
        latents = latents * self.scale_factor
        
        return {
            "latents": latents,
            "input_ids": item["input_ids"].squeeze(0),
            "attention_mask": item["attention_mask"].squeeze(0)
        }

# --- 2. Initialize Components ---
print("Initializing Dataset...")
dataset = VideoLatentDataset(DATA_DIR)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)

print("Loading Tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_NAME)

print("Initializing VideoDiT Model (Flow Matching)...")
# Using the NEW args for Texture-DiT features
model = VideoDiT_models['VideoDiT-S'](
    input_size=INPUT_SIZE,
    patch_size=PATCH_SIZE,
    in_channels=4, 
    vocab_size=tokenizer.vocab_size,
    use_rope=True,
    use_ternary_rope=True,
    first_frame_condition=False,
    full_precision=True,
    optimized_bitlinear=False,
    use_temporal=False,
    use_grid=False,
    use_resampling=False
).to(device)

optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
scaler = GradScaler() 

# --- 3. Training Loop (Flow Matching) ---
print(f"Starting Flow Matching training on {device}...")

model.train()
global_step = 0

for epoch in range(NUM_EPOCHS):
    progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}")
    epoch_loss = 0.0
    optimizer.zero_grad()
    
    for step, batch in enumerate(progress_bar):
        # 1. Prepare Data
        # In Flow Matching, the clean data (latents) is x_1
        x_1 = batch["latents"].to(device)
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        
        cond_y = {"input_ids": input_ids, "attention_mask": attention_mask}
        
        # 2. Forward Pass (Loss Calculation)
        with autocast():
            # The loss function handles sampling t, interpolating x_t, and calculating MSE(v_pred, v_target)
            loss = flow_matching_loss(model, x_1, cond_y)
            loss = loss / GRADIENT_ACCUMULATION_STEPS

        # 3. Backward Pass
        scaler.scale(loss).backward()

        # 4. Optimization
        if (step + 1) % GRADIENT_ACCUMULATION_STEPS == 0:
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            global_step += 1

        current_loss = loss.item() * GRADIENT_ACCUMULATION_STEPS
        epoch_loss += current_loss
        progress_bar.set_postfix({"loss": f"{current_loss:.4f}"})
        
        if step % 100 == 0:
            torch.cuda.empty_cache()

    avg_epoch_loss = epoch_loss / len(dataloader)
    print(f"Epoch {epoch+1} Completed. Average Loss: {avg_epoch_loss:.6f}")
    
    checkpoint_path = os.path.join(MODEL_SAVE_DIR, f"videodit_flow_s_epoch_{epoch+1}.pt")
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': avg_epoch_loss,
    }, checkpoint_path)
    print(f"Saved checkpoint to {checkpoint_path}")